In [1]:
!python --version

Python 3.6.9


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd './drive/My Drive/NLP/day31_32_33'

/content/drive/My Drive/NLP/day31_32_33


In [4]:
#!pip install -q -U pip
#!pip install -q numpy
#!pip install -q pandas
!pip install -q ckiptagger
!pip install -q tqdm
#!pip install -q tensorflow==1.14.0
!pip install -q ipywidgets
#!pip install -q matplotlib

In [5]:
from ckiptagger import data_utils, construct_dictionary, WS, POS

In [ ]:
# Downloads to ./data.zip (2GB) and extracts to ./data/
# data_utils.download_data_url("./") # iis-ckip
# data_utils.download_data_gdown("./") # gdrive-ckip

In [6]:
import pandas as pd
import numpy as np

#from ckiptagger import WS, POS
from tqdm.notebook import tqdm

In [7]:
df_train = pd.read_csv('news_clustering_train.tsv', sep='\t')
df_test = pd.read_csv('news_clustering_test.tsv', sep='\t')

In [8]:
train_titles = {row['index']: row['title'] for _, row in df_train.iterrows()}
train_classes = {row['index']: row['class'] for _, row in df_train.iterrows()}

test_titles = {row['index']: row['title'] for _, row in df_test.iterrows()}
test_classes = {row['index']: row['class'] for _, row in df_test.iterrows()}

In [9]:
all_news_class = ['體育', '財經', '科技', '旅遊', '農業', '遊戲']

# 斷詞 + POS

In [10]:
ws = WS('./data/')
pos = POS('./data/')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


In [11]:
train_title_cuts = {}
for index, title in tqdm(train_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    train_title_cuts[index] = list(zip(word_s[0], word_p[0]))

In [12]:
test_title_cuts = {}
for index, title in tqdm(test_titles.items()):
    word_s = ws([title])
    word_p = pos(word_s)
    test_title_cuts[index] = list(zip(word_s[0], word_p[0]))

# 尋找降維的詞向量：PPMI + SVD

In [13]:
word2index = {}
index2word = {}
n = 0
for index in train_title_cuts:
    for word, flag in train_title_cuts[index]:
        if word in word2index:
            continue
        word2index[word] = n 
        index2word[n] = word
        n += 1

In [14]:
len(word2index)

6690

In [15]:
train_title_cuts[0]

[('亞洲杯', 'Nb'),
 ('奪', 'VC'),
 ('冠賠率', 'Na'),
 ('：', 'COLONCATEGORY'),
 ('日本', 'Nc'),
 ('、', 'PAUSECATEGORY'),
 ('伊朗', 'Nc'),
 ('領銜', 'VB'),
 (' ', 'WHITESPACE'),
 ('中國', 'Nc'),
 ('竟', 'D'),
 ('與', 'P'),
 ('泰國', 'Nc'),
 ('並列', 'VG')]

如果使用one-hot就需要這麼大的維度的詞向量

In [16]:
# 建立Co-Matrix

vocab_size = len(word2index)
co_matrix = np.zeros(shape=(vocab_size, vocab_size), dtype=np.int32)

window_size = 1

for index in train_title_cuts:
    for i , (word, flag) in enumerate(train_title_cuts[index]):
      # current position in sentence:i
      # current word index in matrix
      id = word2index[word]

      # left idx
      if (i - window_size)>0:
        # get left word
        left_word = train_title_cuts[index][i-window_size][0]
        #print(left_word)
        # get left word index in matrix
        left_idx = word2index[left_word]
        co_matrix[id , left_idx] +=1
      
      # right idx
      if (i + window_size) < len(train_title_cuts[index]):
        # get right word
        right_word = train_title_cuts[index][i+window_size][0]
        # get right word index in matrix
        right_idx = word2index[right_word]
        co_matrix[id , right_idx] +=1



In [17]:
co_matrix

array([[0, 2, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
co_matrix.shape

In [18]:
# 建立PPMI

def get_ppmi(co_matrix: np.ndarray, eps: float=1e-8):
  M = np.zeros_like(co_matrix , dtype = np.float32)
  row = M.shape[0]
  col = M.shape[1]

  # ref:https://www.w3resource.com/python-exercises/numpy/basic/numpy-basic-exercise-32.php
  N = np.sum(co_matrix)
  per_count = np.sum(co_matrix , axis = 0)

  total = co_matrix.shape[0]*co_matrix.shape[1]
  cnt = 0

  for i in range(row):
    for j in range(col):
      # numerator:co_matrix[i,j]
      # denominator:c(x),c(y)
      x = per_count[i]
      y = per_count[j]
      M[i,j] = max( np.log2((co_matrix[i,j]*N)/(x*y+eps)) , 0)

      cnt += 1
      if cnt % 10 == 0 or cnt == total:
        print(f"{cnt}/{total} Done")
      
  return M

ppmi = get_ppmi(co_matrix)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: divide by zero encountered in log2


串流輸出內容已截斷至最後 5000 行。
44706110/44756100 Done
44706120/44756100 Done
44706130/44756100 Done
44706140/44756100 Done
44706150/44756100 Done
44706160/44756100 Done
44706170/44756100 Done
44706180/44756100 Done
44706190/44756100 Done
44706200/44756100 Done
44706210/44756100 Done
44706220/44756100 Done
44706230/44756100 Done
44706240/44756100 Done
44706250/44756100 Done
44706260/44756100 Done
44706270/44756100 Done
44706280/44756100 Done
44706290/44756100 Done
44706300/44756100 Done
44706310/44756100 Done
44706320/44756100 Done
44706330/44756100 Done
44706340/44756100 Done
44706350/44756100 Done
44706360/44756100 Done
44706370/44756100 Done
44706380/44756100 Done
44706390/44756100 Done
44706400/44756100 Done
44706410/44756100 Done
44706420/44756100 Done
44706430/44756100 Done
44706440/44756100 Done
44706450/44756100 Done
44706460/44756100 Done
44706470/44756100 Done
44706480/44756100 Done
44706490/44756100 Done
44706500/44756100 Done
44706510/44756100 Done
44706520/44756100 Done
44706530/4475

In [19]:
ppmi

array([[ 0.      ,  9.65118 ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 8.65118 ,  0.      , 11.458535, ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      , 11.458535,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       ...,
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ]], dtype=float32)

In [20]:
# 進行SVD分解，並得到降維的詞向量

from sklearn.decomposition import TruncatedSVD

# 使用`TruncatedSVD`進行降維，降維到dim=1000

svd = TruncatedSVD(n_components = 1000)
word_vectors = svd.fit_transform(ppmi)

In [21]:
word_vectors.shape

(6690, 1000)

# 新的詞向量 + Group mean vector: 測試

In [22]:
excluded_flags = [
    'Nh', 'Nep', 'Nes', 'DE', 'T', 'P', 'V_2', 'SHI',
    'Dfa', 'Dfb', 'Da', 'Di', 'Dk',
    'Caa', 'Cab', 'Cba', 'Cbb',
    'COLONCATEGORY', 'COMMACATEGORY', 'DASHCATEGORY', 'DOTCATEGORY', 'ETCCATEGORY', 'EXCLAMATIONCATEGORY',
    'PARENTHESISCATEGORY', 'PAUSECATEGORY', 'PERIODCATEGORY', 'QUESTIONCATEGORY', 'SEMICOLONCATEGORY',
    'SPCHANGECATEGORY', 'WHITESPACE'
]

In [23]:
train_svd_vectors = {}
for index, pairs in train_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    train_svd_vectors[index] = vector
    

In [24]:
test_svd_vectors = {}
for index, pairs in test_title_cuts.items():
    selected_word_vectors = []
    for word, flag in pairs:
        if word in word2index and flag not in excluded_flags:
            selected_word_vectors.append(word_vectors[word2index[word], :])
    vector = np.sum(selected_word_vectors, axis=0)
    if np.sum(np.square(vector)) == 0:
        continue
    test_svd_vectors[index] = vector

In [25]:
group_vectors = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(train_svd_vectors.items()):
    news_class = train_classes[index]
    group_vectors[news_class].append(vector)

group_mean_vector = {}
for news_class, vectors in group_vectors.items():
    group_mean_vector[news_class] = np.mean(vectors, axis=0)

In [26]:
def cosine_similarity(bow1, bow2):
    len_bow1 = np.sqrt(np.sum(np.square(bow1)))
    len_bow2 = np.sqrt(np.sum(np.square(bow2)))
    return np.sum(bow1 * bow2) / (len_bow1 * len_bow2)

In [27]:
classification = {news_class: [] for news_class in all_news_class}
for index, vector in sorted(test_svd_vectors.items()):
    if np.sum(np.square(vector)) == 0:
        continue

    max_val = -2.0
    max_class = None
    for news_class, ref_vector in group_mean_vector.items():
        val = cosine_similarity(ref_vector, vector)
        if val > max_val:
            max_class = news_class
            max_val = val

    classification[max_class].append(index)

In [28]:
from collections import Counter

for group, ids in classification.items():
    counter = Counter([test_classes[id] for id in ids])
    print('predict', group, ': ', counter)

predict 體育 :  Counter({'體育': 62, '旅遊': 10, '財經': 9, '遊戲': 9, '科技': 5, '農業': 3})
predict 財經 :  Counter({'財經': 63, '科技': 24, '農業': 18, '體育': 8, '遊戲': 8, '旅遊': 6})
predict 科技 :  Counter({'科技': 51, '體育': 14, '財經': 12, '農業': 10, '旅遊': 9, '遊戲': 9})
predict 旅遊 :  Counter({'旅遊': 53, '農業': 11, '科技': 5, '財經': 4, '遊戲': 3, '體育': 2})
predict 農業 :  Counter({'農業': 56, '旅遊': 8, '體育': 4, '遊戲': 4, '財經': 3, '科技': 3})
predict 遊戲 :  Counter({'遊戲': 67, '旅遊': 12, '科技': 11, '財經': 9, '體育': 8, '農業': 1})
